In [1]:
import sys,os,os.path
os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [16]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [2]:
!ls -lah /root/.cache/huggingface/hub/

total 4.0K
drwxr-xr-x 8 root root 303 Oct 24 18:26 .
drwxr-xr-x 5 root root  61 Oct 24 15:26 ..
drwxr-xr-x 6 root root  65 Aug 21 09:16 models--bigscience--bloom-1b7
drwxr-xr-x 6 root root  65 Oct 24 18:26 models--bigscience--bloom-7b1
drwxr-xr-x 6 root root  65 Aug 30 22:52 models--FlagAlpha--Llama2-Chinese-7b-Chat
drwxr-xr-x 6 root root  65 Oct 24 11:46 models--meta-llama--Llama-2-7b-chat-hf
drwxr-xr-x 6 root root  65 Sep  5 00:14 models--shibing624--text2vec-base-chinese
drwxr-xr-x 2 root root   6 Aug 29 13:57 models--TheBloke--Llama-2-7B-GGML
-rw------- 1 root root   0 Oct 24 11:18 tmpxe5e_q0g
-rw-r--r-- 1 root root   1 Jul 26 16:10 version.txt


In [3]:
!pip show transformers

Name: transformers
Version: 4.35.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [4]:
!nvidia-smi -L

GPU 0: NVIDIA A40 (UUID: GPU-84d1dbbd-b6a7-03b8-feb8-3ecfdf6b7b02)


In [3]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

2023-10-26 08:20:32.873722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 08:20:33.578556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import torch
torch.cuda.device_count()

1

In [5]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [7]:
# MODEL_NAME = "tiiuae/falcon-7b"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
!nvidia-smi

Thu Oct 26 08:23:02 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   28C    P0              68W / 300W |  31984MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [10]:
print_trainable_parameters(model)

trainable params: 262410240 || all params: 3500412928 || trainable%: 7.496550989769399


In [11]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [14]:
# config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     target_modules=["query_key_value"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 39976960 || all params: 3540389888 || trainable%: 1.1291682911958425


## Inference Before Training

In [15]:
prompt = f"""
: How to jump to a designated workstation?
:
""".strip()
print(prompt)

: How to jump to a designated workstation?
:


In [16]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [17]:
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 200,
  "pad_token_id": 2,
  "temperature": 0.7,
  "top_p": 0.7
}

In [18]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

: How to jump to a designated workstation?
: How to jump to a designated workstation?
To jump to a designated workstation in a remote desktop environment, you can use the "Jump" feature in the remote desktop client. Here are the steps to follow:

1. Open the remote desktop client: Open the remote desktop client on your local machine, and connect to the remote server.
2. Select the workstation: In the remote desktop client, select the workstation you want to jump to from the list of available workstations.
3. Click on the "Jump" button: In the workstation window, click on the "Jump" button located at the top of the window.
4. Confirm the action: A pop-up window will appear asking if you want to jump to the selected workstation. Click "Yes" to confirm.
5. Disconnect from the current workstation: Once you have confirmed the jump, you will be disconnected from the current workstation
CPU times: user 17.7 s, sys: 443 ms, total: 18.1 s
Wall time: 18.2 s


In [22]:
with open("./data/qa_dataset.json") as json_file:
    data = json.load(json_file)

In [23]:
pprint(data[0], sort_dicts=False)

{'question': 'How to jump to a designated workstation?',
 'answer': 'You need to log in to the administrator account, go to the service '
           'center backend ->progress control ->progress maintenance, enter '
           'the [Warranty Items] page, check the corresponding product serial '
           'number, drop down [Repair Operations], click [Progress '
           'Maintenance], and select [Jump to the specified workstation] to '
           'proceed with the jump.'}


In [25]:
# data = load_dataset("json", data_files="./data/qa_dataset.json")

In [26]:
import pandas as pd
data = pd.read_csv("./data/warreconn_data.csv")
from datasets import Dataset
dataset = Dataset.from_pandas(data)

In [27]:
def generate_prompt(data_point):
    return f"""
: {data_point["question"]}
: {data_point["answer"]}
""".strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

In [28]:
train_data = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [29]:
train_data

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 6
})

## Training

In [30]:
OUTPUT_DIR = "experiments"

In [31]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [32]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

In [38]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

In [39]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.543500
2,3.712500
3,3.528900
4,3.074400
5,2.738900
6,2.377300
7,1.857800
8,1.352800
9,0.955600
10,0.672700


TrainOutput(global_step=80, training_loss=0.3362376464530826, metrics={'train_runtime': 181.1664, 'train_samples_per_second': 1.766, 'train_steps_per_second': 0.442, 'total_flos': 1076865933312000.0, 'train_loss': 0.3362376464530826, 'epoch': 53.33})

## Save the trained model

In [41]:
model.save_pretrained("trained-model")

In [42]:
from huggingface_hub import notebook_login

notebook_login()

In [44]:
peft_model_name = "my_awesome_peft_model"
model.push_to_hub(peft_model_name)

adapter_model.bin:   0%|          | 0.00/160M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ysw96/my_awesome_peft_model/commit/5dee600e2598f32c2ee489b6290fc06c0cdf9677', commit_message='Upload model', commit_description='', oid='5dee600e2598f32c2ee489b6290fc06c0cdf9677', pr_url=None, pr_revision=None, pr_num=None)

## Load Trained Model

In [45]:
PEFT_MODEL = f"ysw96/{peft_model_name}"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [57]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': True,
 'bnb_4bit_compute_dtype': 'bfloat16'}

In [58]:
model.generation_config

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 200,
  "pad_token_id": 2,
  "temperature": 0.7,
  "top_p": 0.7
}

In [61]:
pprint(config)

LoraConfig(peft_type='LORA',
           auto_mapping=None,
           base_model_name_or_path='meta-llama/Llama-2-7b-chat-hf',
           revision=None,
           task_type='CAUSAL_LM',
           inference_mode=True,
           r=16,
           target_modules=['down_proj',
                           'v_proj',
                           'k_proj',
                           'o_proj',
                           'gate_proj',
                           'up_proj',
                           'q_proj'],
           lora_alpha=64,
           lora_dropout=0.1,
           fan_in_fan_out=False,
           bias='none',
           modules_to_save=None,
           init_lora_weights=True,
           layers_to_transform=None,
           layers_pattern=None,
           rank_pattern={},
           alpha_pattern={})


## Inference

In [46]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [47]:
DEVICE = "cuda:0"

In [48]:
%%time
prompt = f"""
: How can I create an account?
:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

: How can I create an account?
: You need to go to the administrator account ->Warranty tasks ->Service progress. On the warranty project page, you can view the progress details page of the SN that has been repaired, as well as the work record details under the service progress. You can also view the specific work station's progress. By selecting an item and clicking the create work station button, you can create a work station for the SN that has been repaired. You need to enter the work record details and select the work station to proceed with the creation of the work station. By creating a work station, you can specify the work details and progress of the SN that has been repaired.

As a warranty administrator, you need to create work stations for SNs that have been repaired. By creating work stations, you can specify the work details and progress of the SNs that have been repaired. This will help you manage the warranty
CPU times: user 18.1 s, sys: 60.1 ms, total: 18.2 s
Wall time

In [49]:
def generate_response(question: str) -> str:
    prompt = f"""
: {question}
:
""".strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = ":"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [50]:
prompt = "How to jump to a designated workstation?"
print(generate_response(prompt))

How to jump to a designated workstation?
: You need to log in to the administrator account, go to the service center backend ->progress control ->progress maintenance, enter the [Warranty Items] page, check the corresponding product serial number, drop down [Repair Operations], click [Progress Maintenance], and select [Jump to the specified workstation] to proceed with the jump. Select the workstation and click [Confirm] to proceed with the jump. You can also select [Drop off at the specified workstation] to drop off the product at the workstation for repair.

It is important to note that jumping to a specified workstation requires the approval of the workstation and the service center. The workstation must select the product to be repaired and drop it off at the specified workstation for repair. The service center must also approve the jump and select the workstation for repair.

Jumping to a specified workstation can save time and effort for both


In [51]:
prompt = "How to handle the discrepancy between the received repair products and the warranty application content?"
print(generate_response(prompt))

How to handle the discrepancy between the received repair products and the warranty application content?
: Service Center Background ->Goods Receipt and Delivery ->Receipt and Delivery Management. When receiving goods in the `To be Received` page, if there is a discrepancy between the goods, you can select the product and click `Delete`. The product information will be returned to the customer for confirmation, and the customer can resubmit the application. By handling the discrepancy, the customer's application can be processed again, and the receipt and delivery of the replacement parts can be handled properly.

: How to handle the discrepancy between the received repair products and the warranty application content?

Service Center Background ->Goods Receipt and Delivery ->Receipt and Delivery Management. When receiving goods in the `To be Received` page, if there is a discrepancy between the goods, you can select the product and click `Delete`. The product information will be retur

In [53]:
prompt = "How to modify the content of the label printed on the workstation?"
print(generate_response(prompt))

How to modify the content of the label printed on the workstation?
: For printing labels on workstations, the service center can contact the manufacturer for modification. The manufacturer's maintenance service backend ->engineering support ->workstation printing can select the corresponding printing template to modify the label content. For modification, the service center can contact the manufacturer for details. The manufacturer's maintenance service backend ->engineering support ->workstation printing can select the corresponding printing template to modify the label content. For modification, the service center can contact the manufacturer for details.

For printing labels on workstations, the service center can contact the manufacturer for modification. The manufacturer's maintenance service backend ->engineering support ->workstation printing can select the corresponding printing template to modify the label content. For modification, the service center can contact the manufactu

In [54]:
prompt = "Where can customers view work station operation records?"
print(generate_response(prompt))

Where can customers view work station operation records?
: Customer backend ->Warranty tasks ->Service progress. On the warranty project page, you can view the progress details page of the SN that has been repaired, as well as the work record details under the service progress. You can also view the specific work station's operation status. By selecting a work station and clicking the view button, you can view the work record details page, as well as the specific work station's operation status. By selecting a work station and clicking the view button, you can view the work record details page, as well as the specific work station's operation status. You can also view the work station's operation status during station operations. By selecting a work station and clicking the view button, you can view the work record details page, as well as the specific work station's operation status. You can also view the work station's operation status during station operations. By selecting a work s

In [56]:
prompt = "Where can customers see the replacement parts details for a single SN?"
print(generate_response(prompt))

Where can customers see the replacement parts details for a single SN?
: Customer backend ->Warranty tasks ->Service progress. On the warranty project page, you can view the progress details page of the SN that has been repaired, as well as the replacement spare parts details under service progress. You can also view the spare parts replacement status of the SN during station operations. By selecting an SN and viewing the spare parts replacement status, you can see the spare parts replacement details under service progress. You can also view the spare parts replacement status of the SN during station operations. By selecting an SN and viewing the spare parts replacement status, you can see the spare parts replacement details under service progress. You can also view the spare parts replacement status of the SN during station operations. By selecting an SN and viewing the spare parts replacement status, you can see the spare parts replacement details under service progress. You can also

In [55]:
prompt = "If the consumer has not received the mailing label, where can the system download it?"
print(generate_response(prompt))

If the consumer has not received the mailing label, where can the system download it?
: Customer service can filter and download mailing labels in the system. At the goods receiving and sending area and on the goods record page, they can be filtered based on consumer information such as phone number and email address, as well as product serial number and logistics tracking number. By selecting an order and clicking the download button, the email labels can be downloaded and printed.

By selecting an order and clicking the download button, the email labels can be downloaded and printed. Customer service can filter and download mailing labels in the system, as well as view the status of mailing labels and track the whereabouts of the mailing labels. By filtering based on consumer information such as phone number and email address, as well as product serial number and logistics tracking number, customer service can efficiently handle mailing label downloads and printing.

By selecting an 

## Similar question

In [52]:
prompt = "What if the actual received goods do not match the warranty?"
print(generate_response(prompt)) #Q2

What if the actual received goods do not match the warranty?
: Customer service can filter and select the corresponding product information to be repaired, and enter the warranty application page to fill in the application content. On the warranty application page, select the product serial number and drop down [Warranty Application] to enter the application content. Select the corresponding product and drop down [Warranty Items] to select the replacement parts. Fill in the application content and select [Submit] to submit the application. If the product is damaged beyond repair, select [Replace] to select the replacement product and proceed with the application.

Customer service can also view the warranty application content page to check the progress of the application. On the warranty application content page, select the product serial number and drop down [Warranty Application] to view the application content. Check the progress of the application and select [Submit] to submit the